In [ ]:
detailed_feedback = {  # mapping from the short feedback options to the statements that go to email
}

In [ ]:
import pandas
from pony import orm
import jinja2
import db
import os
import mimetypes
import subprocess

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [ ]:
db.use_db('EXAM_data/EXAM.sqlite')

In [ ]:
summary = pandas.read_pickle('summary_EXAM.pickle')
grades = pandas.read_pickle('grades_EXAM.pickle')

In [ ]:
# data = {}

# with orm.db_session:

#     possible_feedback = []
#     for problem in db.Problem.select().order_by(db.Problem.id):
#         possible_feedback.extend([
#             problem.name + ':' + fb.text for fb in problem.feedback_options
#         ])

#     for sub in db.Submission.select():
#         a = []
#         for sol in sub.solutions:
#             a.extend([sol.problem.name + ':' + fb.text for fb in sol.feedback])

#         data[sub.student.id] = [
#             option in a for option in possible_feedback
#         ]

#     for problem in db.Problem.select().order_by(db.Problem.id):
#         possible_feedback.append(problem.name + ':' + 'remarks')

#     for sub in db.Submission.select():
#         data[sub.student.id].extend([
#             sol.remarks for sol in sorted(sub.solutions)
#         ])

In [ ]:
student_data = {}

with orm.db_session:
    for sub in db.Submission.select():
        student = [sub.student.first_name, sub.student.last_name, sub.student.email]
        paths = {os.path.split(sol.image_path)[0] for sol in sub.solutions}
        images = {os.path.join(path, image) for path in paths 
                  for image in os.listdir(path) 
                  if image.startswith('page')}
        student.extend(sorted(images))
        student_data[sub.student.id] = student

general_info = pandas.DataFrame(student_data).T

In [ ]:
general_info.columns = ['first', 'last', 'email', 'p1', 'p2', 'p3', 'p4']

In [ ]:
full_info = pandas.merge(general_info, grades, left_index=True, right_index=True)

In [ ]:
template = jinja2.Template("""Dear {{student['first'].split(' ') | first }} {{student['last']}},

Below please find attached the scans of your exam and our feedback.
If you have any questions, don't hesitate to contant us during the exercise classes 
or in the course chat https://chat.quantumtinkerer.tudelft.nl/solidstate

Additionally, you may want to compare your results with the overall performance
of your colleagues over here: http://nbviewer.jupyter.org/url/antonakhmerov.org/misc/minitest1_summary.ipynb
as well as a reference solution http://antonakhmerov.org/misc/minitest1_example_solutions.pdf.

{% for problem_name in student.keys()|sort -%}
{%- if problem_name.startswith('problem ') and ':' not in problem_name -%}
{{problem_name}} (your score: {{student[problem_name]}}):
{% for feedback_opt in student.keys()
   if ':' in feedback_opt 
      and feedback_opt.startswith(problem_name)
      and student[feedback_opt] %}
    * {{ detailed_feedback[feedback_opt] | wordwrap | indent(width=6) }}
{% endfor %}
{% endif -%}
{%- endfor %}

{%- if student['total'] < 77 -%}
Take care: this test was easier than the rest of the course and the final exam will be.
If you found it hard, try more exercises, and probably exams from the past years.
{%- else -%}
You've done well on this test. However don't relax yet, 
it was easier than the rest of the course will be.
{%- endif%}

Best of luck with the rest of the course,
Solid state course team.
""")

template.trim_blocks = template.lstrip_blocks = True
jinja2.environment.trim_blocks = jinja2.environment.lstrip_blocks = True

messages = []
for nr in full_info.index:
    student = full_info.ix[nr]
    filename = '{}.pdf'.format(nr)
    subprocess.call(['convert', student.p1, student.p2, student.p3, student.p4, filename])
    msg = MIMEMultipart()
    msg['Subject'] = 'Your results for the first minitest'
    msg['From'] = 'no-reply@solidstate.tudelft.nl'
    msg['To'] = student.email
    msg['Reply-to'] = 'no-reply@solidstate.tudelft.nl'
    msg.attach(MIMEText(template.render(student=student, detailed_feedback=detailed_feedback), 'plain'))

    ctype, encoding = mimetypes.guess_type(filename)
    if ctype is None or encoding is not None:
        ctype = 'application/octet-stream'
    maintype, subtype = ctype.split('/', 1)
    with open(filename, 'rb') as fp:
        pdf = MIMEBase(maintype, subtype)
        pdf.set_payload(fp.read())
    encoders.encode_base64(pdf)
    # Set the filename parameter
    pdf.add_header('Content-Disposition', 'attachment', filename=filename)
    msg.attach(pdf)
    os.remove(filename)
    messages.append(msg)


In [ ]:
password = getpass.getpass()

In [ ]:
from time import sleep

In [ ]:
import smtplib
import getpass

In [ ]:
with smtplib.SMTP_SSL('smtp-a.tudelft.nl', 465) as s:
    s.login('NETID', password)
    for number, msg in enumerate(messages):
        print(number)
        s.sendmail('TUD_EMAIL', [msg['To']],
                   msg.as_string())
        sleep(2)